In [1]:
!pip install --upgrade pip --quiet
!pip install cvxopt --quiet

In [2]:
#import some useful packages
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import IBMQ, schedule
from qiskit_ibm_provider import IBMProvider
from qiskit.circuit import Gate, Parameter, QuantumCircuit
from qiskit import pulse, schedule
from qiskit.tools.jupyter import *
from qiskit_ibm_provider.job import job_monitor
from qiskit.pulse import Gaussian
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

%config InlineBackend.figure_formats = ['svg']
import time 
import os
import csv

import warnings
warnings.filterwarnings('ignore')

import os
import csv

# IBMQ.save_account('5baae95ab0c4f46ba156eb0909ff9b8b54fffc1c24b7366190b85d6b2293ed9cac1f51cfa4f008890fcdd9f3871a23d904af20a6812bcf9fedb7fd982d968ca1')
provider = IBMQ.load_account()

provider = IBMProvider()
backend = provider.get_backend('ibm_brisbane')

In [2]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.25.3', 'qiskit': '0.44.3', 'qiskit-aer': '0.12.2', 'qiskit-ignis': None, 'qiskit-ibmq-provider': '0.20.2', 'qiskit-nature': '0.6.2', 'qiskit-finance': None, 'qiskit-optimization': '0.5.0', 'qiskit-machine-learning': '0.6.1'}

In [3]:
#get information about backend
backend_defaults = backend.defaults()
backend_properties = backend.properties()
backend_config = backend.configuration()

In [6]:
#hyperparameters
NUM_SHOTS = 1024

qubit = 109
f01 = backend.properties().qubits[qubit][2].value * 1e9
anhar = backend.properties().qubits[qubit][3].value * 1e9
f12 = f01 + anhar

pi = np.pi
cos = np.cos
sin = np.sin
scale_factor = 1e-7

In [7]:
# amp01 = 0.4150600327660328
amp_halfpi_01 = 0.11532683361128736
amp_halfpi_12 = 0.11769405436165835 
amp01 = 2*amp_halfpi_01
amp12 = 2*amp_halfpi_12
dur12 = 96
sig12 = dur12/4
dur01 = 96
sig01 = dur01/4

In [8]:
def S01_sched(phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$R^{(01)}$') as S01_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur01,
                                      amp=amp_halfpi_01,
                                      sigma=sig01,
                                      name=r'$X_{\pi/2}^{(01)}$'), drive_chan)

    return S01_pulse

def S12_sched(phi):
    with pulse.build(backend=backend, default_alignment='sequential',
                     name=r'$R^{(12)}$') as S12_pulse:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        with pulse.phase_offset(-phi, drive_chan):
            pulse.play(pulse.Gaussian(duration=dur12,
                                      amp=amp_halfpi_12,
                                      sigma=sig12,
                                      name=r'$X_{\pi/2}^{(01)}$'), drive_chan)

    return S12_pulse

# Generalize phase tracking protocol with many jobs

In [9]:
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(12)}$') as xpi12_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f12, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur12,
                            amp=amp12,
                            sigma=sig12,
                            name=r'$X_{\pi}^{(12)}$'), drive_chan)

    
#some useful pulse
with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(01)}$') as xpi01_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Gaussian(duration=dur01,
                            amp=amp01,
                            sigma=sig01,
                            name=r'$X_{\pi}^{(01)}$'), drive_chan)



In [15]:
def protocol(order):
    Xpi01 = Gate("$X_{\pi}^{01}$", 1, [])
    Xpi12 = Gate("$X_{\pi}^{12}$", 1, [])

    zero = QuantumCircuit(qubit + 1, 1)
    zero.measure(qubit, 0)

    one = QuantumCircuit(qubit + 1, 1)
    one.x(qubit)
    one.measure(qubit, 0)

    two = QuantumCircuit(qubit + 1, 1)
    two.x(qubit)
    two.append(Xpi12, [qubit])
    two.measure(qubit, 0)
    two.add_calibration(Xpi12, (qubit,), xpi12_sched, [])

    discr_circ = [zero, one, two]
    exp_circs = []
    for i in range(97):
        phi_arr = np.random.default_rng().uniform(0, pi, len(order))
                
        qc = QuantumCircuit(qubit + 1,1)
        for level in order:
            if level == '1':
                name = r"$G^{01}$" + fr"$(\phi_{i+1})$" 
                G01 = Gate(name, 1, [])
                qc.append(G01, [qubit])
                qc.add_calibration(G01, (qubit,), S01_sched(phi_arr[i]), [])
            elif level == '2':
                name = r"$G^{12}$" + fr"$(\phi_{i+1})$" 
                G12 = Gate(name, 1, [])
                qc.append(G12, [qubit])
                qc.add_calibration(G12, (qubit,), S12_sched(phi_arr[i]), [])
            else:
                print('Invalid level')
                break
        qc.measure(qubit,0)
        exp_circs.append(qc)
    exp_circs = discr_circ + exp_circs
    
    return exp_circs, phi_arr

In [18]:
qubit

109

In [12]:
#create a list of random order
import random
def create_order(length):
    order = ''
    for i in range(length):
        order = order + str(random.randint(1,2))
    return order

def create_orders(length):
    orders = set()
    for i in range(20):
        orders.add(create_order(length))
    return orders


In [16]:
orders = []
for i in range(4, 8, 1):
    orders += random.sample(list(create_orders(i)), k=3)
    
#run many jobs with many orders
tg = time.localtime(time.time())
dd = str(tg.tm_mday)
mm = str(tg.tm_mon) 
yy = str(tg.tm_year)
folder = f"{yy}_{mm}_{dd}"
print(folder)

2023_12_30


In [17]:
id_list = []
for order in orders:
    exp_circs, params = protocol(order)
    tg = time.localtime(time.time())
    params_file = f"./{folder}/{order}_params_{yy}_{mm}_{dd}_brisbane.csv"
    np.savetxt(params_file, params, delimiter=',')
    job = backend.run(exp_circs, meas_level=1, meas_return='single', shots=NUM_SHOTS)
    id_list.append(job.job_id())

Traceback (most recent call last):
  Cell In[17], line 3
    exp_circs, params = protocol(order)
  Cell In[15], line 29 in protocol
    qc.add_calibration(G01, (qubit,), S01_sched(phi_arr[i]), [])
IndexError: index 4 is out of bounds for axis 0 with size 4

Use %tb to get the full traceback.


In [ ]:
#save job's id and orders

jobid_file = f"./{folder}/job_id_0_{yy}_{mm}_{dd}_lagos.txt"
with open(jobid_file, 'w') as f:
    for job_id in id_list:
        f.write(job_id)
        f.write('\n')

file = f"./{folder}/order_0_{yy}_{mm}_{dd}_lagos.txt"
with open(file, 'w') as f:
    for order in orders:
        f.write(order)
        f.write('\n')

In [ ]:
phase_shift_job = backend.run(exp_circs, 
                          meas_level=1, 
                          meas_return='single', 
                          shots=1024)
job_monitor(phase_shift_job)
phase_shift_job.job_id()

Job Status: job is queued (7) [Est. wait time: 1 hrs 39 min] 

In [28]:
a = res[0]
a[1] - 2*np.pi

-0.03972665878501669